In [33]:
import pandas as pd
import time
import numpy as np
np.seterr(invalid='ignore')
import nbimporter
from PPP import PPP

newGame = 'Game2Celtics'
Opp = 'Celtics'
Team = 'Heat'

In [34]:
players = ['adebayo', 'butler', 'love', 'lowry', 'martin', 'robinson', 'strus', 'vincent', 'zeller', 'team', 'opp']

In [38]:
def load_data(players, add=False):
    
    new_player_data = {}
    ovr_player_data = {}
    
    for player in players:
        
        if player == 'team':
            
            newplayerFile = ('Team/empty.csv')
            newplayer = pd.read_csv(newplayerFile)
            newplayer = newplayer.set_index('Shot Type')
            
            for df in new_player_data.values():
                
                newplayer = newplayer.add(df)
            
            new_player_data['team'] = newplayer
            newplayer.to_csv(f'../{Opp}/Opp/{newGame[0:5] + Team}Opp.csv')
            newplayer.to_csv(f'Team/{newGame}Team.csv')
            
            if newGame[0:5] == 'Game1':
                ovr_player_data['team'] = newplayer
                newplayer.to_csv(f'!Data/Team.csv')
                
            # Load overall data
            ovrplayerFile = (f'!Data/Team.csv')
            ovrplayer = pd.read_csv(ovrplayerFile)
            ovrplayer = ovrplayer.dropna()
            ovrplayer = ovrplayer.set_index('Shot Type')
            ovrplayer = ovrplayer.astype('int')
            ovr_player_data[player] = ovrplayer
            
            if add == True:

                if newplayer.columns.equals(ovrplayer.columns):
                    ovrplayer = ovrplayer.add(newplayer)
                    ovr_player_data[player] = ovrplayer
                    ovrplayer.to_csv('!Data/Team.csv')
                else:
                    print(f"{player.title()}'s columns do not match")
                
            continue
            
        if player == 'opp':
            
            try:
                newplayerFile = (f'Opp/{newGame}.csv')
                newplayer = pd.read_csv(newplayerFile)
                newplayer = newplayer.dropna()
                newplayer = newplayer.set_index('Shot Type')
                newplayer = newplayer.astype('int')
                new_player_data[player] = newplayer
            
            except(FileNotFoundError):
                print(f'No {player.title()} file found.')
                
            # Load overall data
            ovrplayerFile = (f'!Data/Opp.csv')
            ovrplayer = pd.read_csv(ovrplayerFile)
            ovrplayer = ovrplayer.dropna()
            ovrplayer = ovrplayer.set_index('Shot Type')
            ovrplayer = ovrplayer.astype('int')
            ovr_player_data[player] = ovrplayer
            
            if add == True:

                if newplayer.columns.equals(ovrplayer.columns):
                    ovrplayer = ovrplayer.add(newplayer)
                    ovr_player_data[player] = ovrplayer
                    ovrplayer.to_csv('!Data/Opp.csv')
                else:
                    print(f"{player.title()}'s columns do not match")
                    
            continue

        try:
            newplayerFile = (f'{player.title()}/{newGame + player.title()}.csv')
            newplayer = pd.read_csv(newplayerFile)
            newplayer = newplayer.dropna()
            newplayer = newplayer.set_index('Shot Type')
            newplayer = newplayer.astype('int')
            new_player_data[player] = newplayer
            
        except(FileNotFoundError):
            print(f'No {player.title()} file found.')
            continue

        # Load overall data
        ovrplayerFile = (f'!Data/{player.title()}.csv')
        ovrplayer = pd.read_csv(ovrplayerFile)
        ovrplayer = ovrplayer.dropna()
        ovrplayer = ovrplayer.set_index('Shot Type')
        ovrplayer = ovrplayer.astype('int')
        ovr_player_data[player] = ovrplayer
        
        
        if add == True:
            
            if ovrplayer.equals(newplayer):
                continue

            if newplayer.columns.equals(ovrplayer.columns):
                ovrplayer = ovrplayer.add(newplayer)
                ovr_player_data[player] = ovrplayer
                ovrplayer.to_csv(f'!Data/{player.title()}.csv')
            else:
                print(f"{player.title()}'s columns do not match")
            
    
    return new_player_data, ovr_player_data   

new_player_data, ovr_player_data  = load_data(players, add=False)

In [6]:
newButlerStats = PPP(new_player_data['butler'])
newButlerStats.to_csv(f'Butler/PPP Stats/{newGame}.csv')
newButlerStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.93,32.1,1,40.0,0.82,18.8,25.0,4,25.0,0,N/A,1.0,63.6,50.0,3,66.7,3,33.3
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.87,52.5,1,44.4,0.91,63.2,43.8,15,46.7,1,0.0,0.67,27.3,50.0,1,100.0,1,0.0
TRANSITION,1.0,5.4,1,100.0,0.0,3.8,N/A,0,N/A,0,N/A,2.0,9.1,100.0,1,100.0,0,N/A
Attacking Closeouts,2.0,2.7,0,100.0,2.0,3.8,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,2.0,2.7,0,100.0,2.0,3.8,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [7]:
butlerStats = PPP(ovr_player_data['butler'])
butlerStats.to_csv('!PPP Stats/butlerStats.csv')
butlerStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.96,40.0,4,48.0,0.95,29.9,42.9,14,42.9,0,N/A,0.97,65.6,54.5,6,66.7,5,40.0
PNR Screener,0.0,1.3,0,0.0,0.0,1.8,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.05,42.7,2,46.7,0.88,48.8,40.0,22,40.9,3,33.3,1.83,27.3,80.0,1,100.0,4,75.0
TRANSITION,1.0,2.6,1,100.0,0.0,1.8,N/A,0,N/A,0,N/A,2.0,4.5,100.0,1,100.0,0,N/A
Attacking Closeouts,1.0,2.6,0,50.0,1.0,3.6,50.0,2,50.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,2.5,2.6,0,100.0,2.5,3.6,100.0,1,100.0,1,100.0,N/A,0.0,N/A,0,N/A,0,N/A


In [8]:
newAdebayoStats = PPP(new_player_data['adebayo'])
newAdebayoStats.to_csv(f'Adebayo/PPP Stats/{newGame}.csv')
newAdebayoStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.45,9.5,0,0.0,1.45,14.5,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,2.0,6.9,0,100.0,2.0,10.5,100.0,2,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,1.87,20.3,0,60.0,N/A,0.0,N/A,0,N/A,0,N/A,1.87,58.8,60.0,0,N/A,5,60.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.07,57.8,0,46.7,0.85,61.9,30.0,10,30.0,0,N/A,1.6,50.0,80.0,4,100.0,1,0.0
TRANSITION,1.0,6.9,0,50.0,1.0,10.5,50.0,2,50.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [9]:
adebayoStats = PPP(ovr_player_data['adebayo'])
adebayoStats.to_csv('!PPP Stats/adebayoStats.csv')
adebayoStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.37,6.7,0,0.0,1.37,10.1,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,1.48,13.8,1,83.3,1.24,17.9,80.0,5,80.0,0,N/A,3.0,5.6,100.0,0,N/A,1,100.0
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,1.4,14.6,0,42.9,N/A,0.0,N/A,0,N/A,0,N/A,1.4,43.8,42.9,0,N/A,7,42.9
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.21,45.9,0,56.5,1.01,49.3,43.8,16,43.8,0,N/A,1.71,38.9,85.7,6,100.0,1,0.0
TRANSITION,1.5,11.1,1,80.0,1.0,11.1,66.7,3,66.7,0,N/A,2.5,11.1,100.0,1,100.0,1,100.0
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [10]:
newLoveStats = PPP(new_player_data['love'])
newLoveStats.to_csv(f'Love/PPP Stats/{newGame}.csv')
newLoveStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,0.0,40.0,1,0.0,0.0,50.0,0.0,1,0.0,0,N/A,0.0,33.3,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.0,40.0,0,50.0,0.0,50.0,0.0,1,0.0,0,N/A,2.0,33.3,100.0,1,100.0,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,3.0,20.0,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,3.0,33.3,100.0,0,N/A,1,100.0


In [11]:
loveStats = PPP(ovr_player_data['love'])
loveStats.to_csv('!PPP Stats/loveStats.csv')
loveStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,0.0,14.3,1,0.0,0.0,14.3,0.0,1,0.0,0,N/A,0.0,14.3,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,7.1,1,N/A,N/A,0.0,N/A,0,N/A,0,N/A,0.0,14.3,N/A,0,N/A,0,N/A
DHO Screener,0.0,7.1,0,0.0,0.0,14.3,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.0,14.3,0,50.0,0.0,14.3,0.0,1,0.0,0,N/A,2.0,14.3,100.0,1,100.0,0,N/A
TRANSITION,2.67,21.4,0,100.0,3.0,28.6,100.0,0,N/A,2,100.0,2.0,14.3,100.0,1,100.0,0,N/A
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,1.5,14.3,1,100.0,0.0,14.3,N/A,0,N/A,0,N/A,3.0,14.3,100.0,0,N/A,1,100.0


In [12]:
newLowryStats = PPP(new_player_data['lowry'])
newLowryStats.to_csv(f'Lowry/PPP Stats/{newGame}.csv')
newLowryStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.33,30.0,0,66.7,0.0,16.7,0.0,0,N/A,1,0.0,2.0,50.0,100.0,2,100.0,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.0,20.0,2,N/A,0.0,33.3,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,0.67,30.0,0,33.3,0.0,16.7,0.0,1,0.0,0,N/A,1.0,50.0,50.0,2,50.0,0,N/A
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,0.0,20.0,0,0.0,0.0,33.3,0.0,0,N/A,2,0.0,N/A,0.0,N/A,0,N/A,0,N/A


In [13]:
lowryStats = PPP(ovr_player_data['lowry'])
lowryStats.to_csv('!PPP Stats/lowryStats.csv')
lowryStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.79,48.3,1,38.5,0.88,47.1,37.5,5,40.0,3,33.3,0.67,50.0,40.0,4,50.0,1,0.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.24,22.2,2,75.0,0.5,23.5,50.0,2,50.0,0,N/A,2.46,20.3,100.0,1,100.0,1,100.0
TRANSITION,1.83,20.7,0,66.7,1.5,11.8,50.0,1,0.0,1,100.0,2.0,33.3,75.0,2,50.0,2,100.0
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,1.0,10.3,0,33.3,1.0,17.6,33.3,0,N/A,3,33.3,N/A,0.0,N/A,0,N/A,0,N/A


In [14]:
newMartinStats = PPP(new_player_data['martin'])
newMartinStats.to_csv(f'Martin/PPP Stats/{newGame}.csv')
newMartinStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,2.0,11.8,0,100.0,2.0,11.8,100.0,2,100.0,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
PNR Screener,1.0,11.8,0,50.0,1.0,11.8,50.0,1,100.0,1,0.0,N/A,N/A,N/A,0,N/A,0,N/A
DHO Ball Handler,3.0,5.9,0,100.0,3.0,5.9,100.0,0,N/A,1,100.0,N/A,N/A,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
ISOLATION,0.0,5.9,0,0.0,0.0,5.9,0.0,1,0.0,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
TRANSITION,1.0,11.8,1,100.0,1.0,11.8,100.0,1,100.0,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
Attacking Closeouts,2.0,17.6,0,100.0,2.0,17.6,100.0,3,100.0,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
Catch & Shoot,1.2,29.4,0,40.0,1.2,29.4,40.0,0,N/A,5,40.0,N/A,N/A,N/A,0,N/A,0,N/A


In [15]:
martinStats = PPP(ovr_player_data['martin'])
martinStats.to_csv('!PPP Stats/martinStats.csv')
martinStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,2.0,6.2,0,100.0,2.0,7.1,100.0,2,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,0.67,9.4,0,33.3,0.67,10.7,33.3,1,100.0,2,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,3.0,3.1,0,100.0,3.0,3.6,100.0,0,N/A,1,100.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.0,6.2,0,0.0,0.0,3.6,0.0,1,0.0,0,N/A,0.0,25.0,0.0,0,N/A,1,0.0
TRANSITION,0.8,15.6,1,50.0,0.8,17.9,50.0,3,66.7,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,2.29,21.9,0,100.0,2.0,17.9,100.0,5,100.0,0,N/A,3.0,50.0,100.0,0,N/A,2,100.0
Catch & Shoot,1.64,34.4,0,54.5,1.5,35.7,50.0,0,N/A,10,50.0,3.0,25.0,100.0,0,N/A,1,100.0


In [16]:
newRobinsonStats = PPP(new_player_data['robinson'])
newRobinsonStats.to_csv(f'Robinson/PPP Stats/{newGame}.csv')
newRobinsonStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,3.0,10.0,0,100.0,3.0,11.1,100.0,0,N/A,1,100.0,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,0.0,10.0,0,0.0,0.0,11.1,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,1.5,40.0,0,50.0,1.5,44.4,50.0,0,N/A,4,50.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.0,10.0,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,100.0,0.0,1,0.0,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [17]:
robinsonStats = PPP(ovr_player_data['robinson'])
robinsonStats.to_csv('!PPP Stats/robinsonStats.csv')
robinsonStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.5,16.7,0,50.0,1.5,18.2,50.0,0,N/A,2,50.0,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,0.0,8.3,0,0.0,0.0,9.1,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,1.2,41.7,0,40.0,1.2,45.5,40.0,0,N/A,5,40.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.0,8.3,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,100.0,0.0,1,0.0,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [18]:
newStrusStats = PPP(new_player_data['strus'])
newStrusStats.to_csv(f'Strus/PPP Stats/{newGame}.csv')
newStrusStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.67,42.9,0,66.7,2.5,33.3,100.0,1,100.0,1,100.0,0.0,100.0,0.0,0,N/A,1,0.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,1.7,25.1,0,N/A,1.7,29.3,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,1.5,28.6,0,50.0,1.5,33.3,50.0,0,N/A,2,50.0,N/A,0.0,N/A,0,N/A,0,N/A


In [19]:
strusStats = PPP(ovr_player_data['strus'])
strusStats.to_csv('!PPP Stats/strusStats.csv')
strusStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,2.17,33.3,0,83.3,2.6,33.3,100.0,2,100.0,3,100.0,0.0,33.3,0.0,0,N/A,1,0.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,11.1,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,66.7,0.0,1,0.0,1,0.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,1.86,20.9,0,100.0,1.86,25.1,100.0,2,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.0,5.6,0,0.0,0.0,6.7,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,1.5,22.2,0,50.0,1.5,26.7,50.0,0,N/A,4,50.0,N/A,0.0,N/A,0,N/A,0,N/A


In [20]:
newVincentStats = PPP(new_player_data['vincent'])
newVincentStats.to_csv(f'Vincent/PPP Stats/{newGame}.csv')
newVincentStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.25,36.4,0,50.0,1.25,44.4,50.0,1,100.0,3,33.3,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,1.06,17.1,1,N/A,1.06,20.9,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.69,26.2,1,0.0,1.06,20.9,0.0,1,0.0,0,N/A,0.0,50.0,N/A,0,N/A,0,N/A
TRANSITION,0.0,9.1,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,50.0,0.0,0,N/A,1,0.0
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,0.0,18.2,0,0.0,0.0,22.2,0.0,0,N/A,2,0.0,N/A,0.0,N/A,0,N/A,0,N/A


In [21]:
vincentStats = PPP(ovr_player_data['vincent'])
vincentStats.to_csv('!PPP Stats/vincentStats.csv')
vincentStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.29,31.8,0,57.1,1.4,31.2,60.0,2,100.0,3,33.3,1.0,33.3,50.0,1,100.0,1,0.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.69,13.1,1,0.0,0.69,18.0,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.41,22.2,2,0.0,0.69,18.0,0.0,1,0.0,1,0.0,0.0,33.3,N/A,0,N/A,0,N/A
TRANSITION,1.5,9.1,0,50.0,N/A,0.0,N/A,0,N/A,0,N/A,1.5,33.3,50.0,0,N/A,2,50.0
Attacking Closeouts,2.27,4.0,0,N/A,2.27,5.5,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,1.5,18.2,0,50.0,1.5,25.0,50.0,0,N/A,4,50.0,N/A,0.0,N/A,0,N/A,0,N/A


In [22]:
newZellerStats = PPP(new_player_data['zeller'])
newZellerStats.to_csv(f'Zeller/PPP Stats/{newGame}.csv')
newZellerStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,0.0,25.0,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,100.0,0.0,0,N/A,1,0.0
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.0,25.0,0,0.0,0.0,33.3,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [23]:
zellerStats = PPP(ovr_player_data['zeller'])
zellerStats.to_csv('!PPP Stats/zellerStats.csv')
zellerStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,0.0,37.5,1,0.0,0.0,20.0,0.0,1,0.0,0,N/A,0.0,66.7,0.0,0,N/A,1,0.0
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.0,12.5,0,0.0,0.0,20.0,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [36]:
newTeamStats = PPP(new_player_data['team'])
newTeamStats.to_csv(f'Team/PPP Stats/{newGame}.csv')
newTeamStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.3,20.9,1,54.2,1.42,17.8,53.3,9,55.6,6,50.0,1.1,30.3,55.6,5,80.0,4,25.0
PNR Screener,0.75,6.1,1,42.9,1.0,6.1,50.0,4,75.0,2,0.0,0.0,6.1,0.0,0,N/A,1,0.0
DHO Ball Handler,1.6,5.2,1,60.0,1.6,6.9,60.0,0,N/A,5,60.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,1.87,4.5,0,60.0,N/A,0.0,N/A,0,N/A,0,N/A,1.87,17.8,60.0,0,N/A,5,60.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.88,33.4,4,43.2,0.79,34.4,34.5,28,35.7,1,0.0,1.2,30.3,75.0,6,100.0,2,0.0
TRANSITION,0.94,8.9,2,50.0,0.9,7.8,50.0,4,50.0,0,N/A,1.0,12.1,50.0,3,66.7,1,0.0
Attacking Closeouts,1.33,4.5,0,66.7,1.6,5.1,80.0,5,80.0,0,N/A,0.0,3.0,0.0,1,0.0,0,N/A
Catch & Shoot,1.08,9.8,0,38.5,0.92,12.1,33.3,1,100.0,11,27.3,3.0,3.0,100.0,0,N/A,1,100.0


In [39]:
newOppStats = PPP(new_player_data['opp'])
newOppStats.to_csv(f'Opp/PPP Stats/{newGame}.csv')
newOppStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.17,15.7,3,52.9,1.02,8.5,50.0,5,60.0,1,0.0,1.26,29.8,54.5,4,75.0,7,42.9
PNR Screener,1.74,7.3,2,85.7,2.37,6.8,100.0,3,100.0,2,100.0,0.75,8.3,50.0,0,N/A,2,50.0
DHO Ball Handler,2.5,1.4,0,100.0,3.0,1.1,100.0,0,N/A,1,100.0,2.0,2.1,100.0,1,100.0,0,N/A
DHO Screener,2.0,0.7,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,2.0,2.1,100.0,1,100.0,0,N/A
DBL Ball Handler,2.0,0.7,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,2.0,2.1,100.0,1,100.0,0,N/A
DBL Screener,2.0,0.7,0,100.0,2.0,1.1,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.03,19.9,3,45.5,1.08,21.8,47.1,12,41.7,5,60.0,0.9,16.2,40.0,2,100.0,3,0.0
TRANSITION,0.94,20.5,3,38.1,0.88,18.4,30.8,6,50.0,7,14.3,1.02,24.5,50.0,4,100.0,4,0.0
Attacking Closeouts,1.23,11.6,1,57.1,1.01,12.8,50.0,6,83.3,4,0.0,1.8,9.2,75.0,1,100.0,3,66.7
Catch & Shoot,0.74,13.5,2,29.4,0.71,18.3,26.7,0,N/A,15,26.7,1.0,4.2,50.0,1,100.0,1,0.0


In [37]:
teamStats = PPP(ovr_player_data['team'])
teamStats.to_csv('!PPP Stats/teamStats.csv')
teamStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.14,24.4,5,51.8,1.3,22.0,54.1,26,53.8,11,54.5,0.85,30.4,47.4,11,63.6,8,25.0
PNR Screener,0.75,6.5,3,42.9,0.74,7.0,41.7,9,55.6,3,0.0,0.75,5.2,50.0,0,N/A,2,50.0
DHO Ball Handler,0.93,4.4,2,33.3,1.24,4.6,42.9,0,N/A,7,42.9,0.0,3.9,0.0,1,0.0,1,0.0
DHO Screener,1.24,3.3,0,37.5,0.0,0.5,0.0,0,N/A,1,0.0,1.4,10.2,42.9,0,N/A,7,42.9
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.05,27.2,6,49.2,0.85,28.0,38.3,43,39.5,4,25.0,1.59,25.2,81.2,9,100.0,7,57.1
TRANSITION,1.59,10.3,3,73.9,1.35,9.2,69.2,9,66.7,4,75.0,2.0,13.0,80.0,5,80.0,5,80.0
Attacking Closeouts,1.55,4.8,0,66.7,1.42,5.1,66.7,9,66.7,0,N/A,2.0,3.9,66.7,1,0.0,2,100.0
Catch & Shoot,1.58,9.6,1,56.0,1.46,12.4,52.2,1,100.0,22,50.0,3.0,2.6,100.0,0,N/A,2,100.0


In [40]:
oppStats = PPP(ovr_player_data['opp'])
oppStats.to_csv('!PPP Stats/oppStats.csv')
oppStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.19,16.5,6,55.3,1.06,12.2,52.9,11,63.6,6,33.3,1.31,24.5,57.1,10,70.0,11,45.5
PNR Screener,1.69,4.8,2,80.0,1.96,4.8,85.7,5,80.0,2,100.0,1.2,4.9,66.7,0,N/A,3,66.7
DHO Ball Handler,1.67,1.0,0,66.7,1.5,1.0,50.0,0,N/A,2,50.0,2.0,1.0,100.0,1,100.0,0,N/A
DHO Screener,1.0,0.7,0,50.0,N/A,0.0,N/A,0,N/A,0,N/A,1.0,1.9,50.0,1,100.0,1,0.0
DBL Ball Handler,1.2,1.7,0,60.0,0.67,1.6,33.3,3,33.3,0,N/A,2.0,1.9,100.0,2,100.0,0,N/A
DBL Screener,2.0,0.7,0,100.0,2.0,1.0,100.0,2,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.05,20.9,5,46.9,1.18,17.6,51.9,21,47.6,6,66.7,0.89,27.1,40.9,11,72.7,11,9.1
TRANSITION,1.19,18.5,7,52.6,1.09,16.1,45.5,13,53.8,9,33.3,1.32,22.8,62.5,8,87.5,8,37.5
Attacking Closeouts,1.1,13.2,4,51.7,1.07,16.0,52.2,19,63.2,4,0.0,1.2,8.1,50.0,1,100.0,5,40.0
Catch & Shoot,0.7,12.5,2,25.7,0.71,17.6,25.0,0,N/A,32,25.0,0.67,2.9,33.3,1,100.0,2,0.0
